In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import numpy as np
import pickle

In [2]:
def moving_average(x, w):
        moving_avg = np.convolve(x, np.ones(w), 'valid') / w
        padding = np.full_like(np.empty(w), np.nan)
        return np.insert(moving_avg, 0, padding)

def moving_avg_diff(short, long):
        return (short - long) / long


def get_X_y(df, binary_output=True):
    
    min_period = 30
    
    percent_change = (df["Close"] - df["Close"].shift(1)) / df["Close"].shift(1)
    print(percent_change[:5])

    y = percent_change.copy()
    
    print(len(y))
    
    ## TEMP
    if binary_output == True:
        y = np.zeros(percent_change.shape)
        y[np.where(percent_change > 0)] = 1
    else:
        y = y.to_numpy()
    
    moving_average_1_day = moving_average(df["Close"], 1)
    moving_average_2_day = moving_average(df["Close"], 2)
    moving_average_3_day = moving_average(df["Close"], 3)
    # moving_average_4_day = moving_average(df["Close"], 4)
    # moving_average_5_day = moving_average(df["Close"], 5)
    # moving_average_6_day = moving_average(df["Close"], 6)
    moving_average_7_day = moving_average(df["Close"], 7)
    moving_average_14_day = moving_average(df["Close"], 14)
    moving_average_30_day = moving_average(df["Close"], 30)
    
    mv_1d_2d = moving_avg_diff(moving_average_1_day, moving_average_2_day)
    mv_1d_3d = moving_avg_diff(moving_average_1_day, moving_average_3_day)
    # mv_2d_4d = moving_avg_diff(moving_average_1_day, moving_average_3_day)
    mv_3d_7d = moving_avg_diff(moving_average_3_day, moving_average_7_day)
    mv_7d_14d = moving_avg_diff(moving_average_7_day, moving_average_14_day)
    mv_7d_30d = moving_avg_diff(moving_average_7_day, moving_average_30_day)
        
    X = np.stack((mv_1d_2d, mv_1d_3d, mv_3d_7d, mv_7d_14d, mv_7d_30d), axis=1)
    X = X[min_period:len(X)-1]
    y = y[min_period:]
    
    print(len(X), len(y))
    
    return (X, y)

In [3]:
df = pd.read_csv("../../data/clean/bitcoin_prices.csv")
df = df.iloc[::24, :]

df["Time"] = pd.to_datetime(df['Time'],unit='s')
df.set_index(pd.DatetimeIndex(df['Time']), inplace=True, drop=True)
start_date = datetime.datetime(2018,7,1)
end_date = datetime.datetime(2020,1,1)
df = df[df["Time"] > start_date]
df = df[df["Time"] < end_date]

offset = 30 # TODO: Refactor to output ox get_features
X, y = get_X_y(df)
df = df[offset:]
for i in range(0, len(X[0,:])):
    df["X_" + str(i)] = X[:,i]
df.describe()
print(df.columns)

Time
2018-07-01 21:00:00         NaN
2018-07-02 21:00:00    0.045127
2018-07-03 21:00:00   -0.006320
2018-07-04 21:00:00    0.012858
2018-07-05 21:00:00   -0.030138
Name: Close, dtype: float64
549
519 519
Index(['Unnamed: 0', 'Time', 'Low', 'High', 'Open', 'Close', 'Volume', 'X_0',
       'X_1', 'X_2', 'X_3', 'X_4'],
      dtype='object')


In [4]:
class CommInfoFractional(bt.CommissionInfo):
    def getsize(self, price, cash):
        '''Returns fractional size for cash operation @price'''
        return self.p.leverage * (cash / price)

In [5]:
# Create a Stratey
class NNStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.X_0 = self.datas[0].X_0
        self.X_1 = self.datas[0].X_1
        self.X_2 = self.datas[0].X_2
        self.X_3 = self.datas[0].X_3
        self.X_4 = self.datas[0].X_4

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        
        filename = "../../wip-models/pickles/neural_net_v2_daily_price"
        self.clf = pickle.load(open(filename, 'rb'))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            if order.status == order.Canceled:
                self.log("Order Cancelled")
            elif order.status == order.Margin:
                self.log("Order Margin")
            else:
                self.log("Order Rejected")
            self.log(order.status)

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
#         self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        print("Cash available: " + str(self.broker.get_cash()))
        if self.order:
            return
    
        X = [[self.X_0[0], self.X_1[0], self.X_2[0], self.X_3[0], self.X_4[0]]]
        pred = self.clf.predict(X)[0]
#         if not self.position:
        print(pred, "no position", "new buy" if pred == 1 else "sit out")
        if pred == 1:
            self.log('BUY CREATE, %.2f' % self.dataclose[0])

            # Cancel existing orders 
            self.broker.cancel(self.order)

            self.buy(exectype=bt.Order.Market, size=(self.broker.get_cash() * 0.999 / self.data.close[0]))
                
        elif self.position and pred == 0:
            print(pred, "have active position", "sell")
            self.log('SELL CREATE, %.2f' % self.dataclose[0])
            # size = self.broker.get_value() / self.dataclose[0]
            self.sell(size=self.position.size)
        else:
            print(pred, "have active position", "hold positition")

class PandasData(bt.feeds.PandasData):
    params = (
        ('X_0', 'X_0'),
        ('X_1', 'X_1'),
        ('X_2', 'X_2'),
        ('X_3', 'X_3'),
        ('X_4', 'X_4'),
    )
    lines = ('X_0','X_1', 'X_2', 'X_3', 'X_4')

if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(NNStrategy)

    data = PandasData(dataname=df)
    cerebro.adddata(data)

    cerebro.broker.setcash(10000)

    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

    cerebro.broker.addcommissioninfo(CommInfoFractional())

    cerebro.broker.set_coc(True) # cheat_on_close to true (allow buying at same price to avoid order failing with margin)

    
    ## Include commission to see algo's returns with Coinbase Fees
    cerebro.broker.setcommission(commission=0.00075, margin=None) 
    
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe')
    cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annualreturn')
    cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='tradeanalyzer')

    cerebro.addobserver(bt.observers.DrawDown)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    strats = cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('HODL final value:. {0:.2f} Open: {1:.2f}, Close: {2:.2f}'.format(df.iloc[-1]["Close"] * 10000 / df.iloc[0]["Open"], df.iloc[0]["Open"], df.iloc[-1]["Close"]))

Starting Portfolio Value: 10000.00


/usr/local/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.24.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.24.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Cash available: 10000.0
1.0 no position new buy
2018-07-31, BUY CREATE, 7706.44
2018-08-01, BUY EXECUTED, Price: 7706.44, Cost: 9990.00, Comm 7.49
Cash available: 2.5075000000000003
1.0 no position new buy
2018-08-01, BUY CREATE, 7472.99
2018-08-02, BUY EXECUTED, Price: 7472.99, Cost: 2.50, Comm 0.00
Cash available: 0.0006287556250001063
1.0 no position new buy
2018-08-02, BUY CREATE, 7529.00
2018-08-03, BUY EXECUTED, Price: 7529.00, Cost: 0.00, Comm 0.00
Cash available: 1.5766047296876277e-07
1.0 no position new buy
2018-08-03, BUY CREATE, 7420.20
2018-08-04, BUY EXECUTED, Price: 7420.20, Cost: 0.00, Comm 0.00
Cash available: 3.953336359690431e-11
1.0 no position new buy
2018-08-04, BUY CREATE, 6991.20
2018-08-05, BUY EXECUTED, Price: 6991.20, Cost: 0.00, Comm 0.00
Cash available: 9.912990921921378e-15
1.0 no position new buy
2018-08-05, BUY CREATE, 7054.04
2018-08-06, BUY EXECUTED, Price: 7054.04, Cost: 0.00, Comm 0.00
Cash available: 2.485682473671292e-18
1.0 no position new buy
201

In [6]:
print('Returns:', strats[0].analyzers.returns.pprint())
print('Sharpe Ratio:', strats[0].analyzers.mysharpe.pprint())
print('Annual Return:', strats[0].analyzers.annualreturn.pprint())
print('Drawdown:', strats[0].analyzers.drawdown.pprint())
print('Trade Analyzer:', strats[0].analyzers.tradeanalyzer.pprint())

OrderedDict([('rtot', 0.042221066042382946),
             ('ravg', 8.135080162308853e-05),
             ('rnorm', 0.02071197857853155),
             ('rnorm100', 2.0711978578531554)])
Returns: None
OrderedDict([('sharperatio', 0.23386537152444845)])
Sharpe Ratio: None
OrderedDict([(2018, -0.3309342984983613), (2019, 0.5590771585656527)])
Annual Return: None
AutoOrderedDict([('len', 175),
                 ('drawdown', 46.23080856721657),
                 ('moneydown', 8968.800411947692),
                 ('max',
                  AutoOrderedDict([('len', 250),
                                   ('drawdown', 50.90220878666876),
                                   ('moneydown', 9875.054434126)]))])
Drawdown: None
AutoOrderedDict([('total',
                  AutoOrderedDict([('total', 63),
                                   ('open', 0),
                                   ('closed', 63)])),
                 ('streak',
                  AutoOrderedDict([('won',
                               

In [8]:
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]